In [1]:
import numpy as np
import pandas as pd
import pickle
import csv
import os
from torch_geometric.data import Data

import torch
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm

import networkx as nx
import matplotlib.pyplot as plt
    

/home/valdemar/[/home/desktop/anaconda3]/envs/master_env/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370156314/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
pickle_in1 = open("defect_matrices/dict.defects","rb")
pickle_in2 = open("defect_matrices/dict.eq_distr","rb")

defects = []
eq_distr = []

while 1:
    try:
        defects.append(pickle.load(pickle_in1))
        eq_distr.append(pickle.load(pickle_in2))
    except EOFError:
        break

print(len(defects))
print(len(eq_distr))

100
100


In [19]:
def combine_defect_matrices(qubit_matrix):
    
    a = np.array(qubit_matrix[0])
    b = np.array(qubit_matrix[1])
    a_reshaped = np.zeros((5,5))
    b_reshaped = np.zeros((5,5))

    a_reshaped[:a.shape[0],:a.shape[1]] = a
    b_reshaped[:b.shape[0],:b.shape[1]] = b

    for ix in range(4):
        if ix ==0:
            a_reshaped = np.insert(a_reshaped,ix+1,0,axis=1)
            a_reshaped = np.insert(a_reshaped,ix,0,axis=0)
        
            b_reshaped = np.insert(b_reshaped,ix,0,axis=1)
            b_reshaped = np.insert(b_reshaped,ix+1,0,axis=0)
        else:
            a_reshaped = np.insert(a_reshaped,(ix*2)+1,0,axis=1)
            a_reshaped = np.insert(a_reshaped,ix*2,0,axis=0)
        
            b_reshaped = np.insert(b_reshaped,ix*2,0,axis=1)
            b_reshaped = np.insert(b_reshaped,(ix*2)+1,0,axis=0)
        
    combine = a_reshaped+b_reshaped
    #print(combine)
    return combine

def manhattan_dist(positions,nbr_of_defects):
    manhattan_distances = np.zeros((nbr_of_defects,nbr_of_defects))
    for ix in range(nbr_of_defects):
        for jx in range(nbr_of_defects):
            manhattan_distances[ix][jx] = abs(positions[ix,0]-positions[jx,0])+abs(positions[ix,1]-positions[jx,1])

    return manhattan_distances

def distance_to_boundary(combined_matrix):
    
    size = 5
    defect_positions = np.argwhere(combined_matrix)
    nbr_of_defects=len(defect_positions)
    new_pos= np.zeros((nbr_of_defects,2))
    
    for ix in range(nbr_of_defects):
        if defect_positions[ix][0]%2 == 0 :
            aa = defect_positions[ix,1]*0.5
            new_pos[ix] = [defect_positions[ix,0]*0.5,aa]
        else:
            bb = defect_positions[ix,0]*0.5
            new_pos[ix] = [bb,defect_positions[ix,1]*0.5]

    ##################
    
    manhattan_distances = manhattan_dist(new_pos,nbr_of_defects)
    
    graph = nx.from_numpy_matrix(manhattan_distances)
    graph = nx.to_directed(graph)
    layout= nx.spring_layout(graph)
    #nx.draw(graph,with_labels=True)
    #nx.draw_networkx_edge_labels(graph,pos=layout,font_size=8)

    file_2 = open('defect_matrices/edgelist.txt','ab')        
    nx.write_edgelist(graph, file_2)
    #new_line = '\n'
    #file_2.write(new_line.encode('utf-8'))

    file_3 = open('defect_matrices/graph_info.txt','a')        
    file_3.write(str(nbr_of_defects)+'\n')
    #file_2.write(new_line.encode('utf-8'))

    
    ##################
    
    order_list_vertex = []
    order_list_plaquette = []
    for ix in range(nbr_of_defects):
        if (round(new_pos[ix,0],2)-int(new_pos[ix,0]) != 0.):
            #print(new_pos[ix,0])
            order_list_vertex.append(ix)
        if (round(new_pos[ix,1],2)-int(new_pos[ix,1]) != 0.):
            #print(new_pos[ix,1])
            order_list_plaquette.append(ix)

    order_array_plaquette = np.array(order_list_plaquette)
    order_array_vertex = np.array(order_list_vertex)

    #print(len(order_array_plaquette))
    #print(len(order_array_vertex))
    #print(order_array_vertex)

    #print(new_pos[order_array_vertex])
    #print(new_pos[order_array_plaquette])
    
    if len(order_array_plaquette) == 0:
        distance_vertex_boundary = np.zeros(len(order_array_vertex))
        for ix in range(len(order_array_vertex)):
            dist = new_pos[order_array_vertex[ix]]
            if dist[0] >= 2.5:
                dist[0] = -(size-dist[0])
    
            distance_vertex_boundary[ix] = dist[0]
    
    
    
        boundary_vertex = np.around(distance_vertex_boundary+0.1)*0.5
    
        boundary_v = np.zeros((len(boundary_vertex),2))
        boundary_v[:,0] = boundary_vertex

        node_features = np.zeros((nbr_of_defects,2))
        node_features[order_array_vertex] = boundary_v
    
    
    ############
    
    elif len(order_array_vertex) == 0:
        distance_plaquette_boundary = np.zeros(len(order_array_plaquette))

        for ix in range(len(order_array_plaquette)):
            dist = new_pos[order_array_plaquette[ix]]
            if dist[1] >= 2.5:
                dist[1] = -(size-dist[1])
    
            distance_plaquette_boundary[ix] = dist[1]
    

        boundary_plaquette = np.around(distance_plaquette_boundary+0.1)*0.5

    
        boundary_p = np.ones((len(boundary_plaquette),2))
        boundary_p[:,0] = boundary_plaquette
    
        node_features = np.zeros((nbr_of_defects,2))
        node_features[order_array_plaquette] = boundary_p
    
    
        
    ##########
    
    else:
        distance_vertex_boundary = np.zeros(len(order_array_vertex))
        distance_plaquette_boundary = np.zeros(len(order_array_plaquette))

        for ix in range(len(order_array_vertex)):
            dist = new_pos[order_array_vertex[ix]]
            if dist[0] >= 2.5:
                dist[0] = -(size-dist[0])
    
            distance_vertex_boundary[ix] = dist[0]
    
        for ix in range(len(order_array_plaquette)):
            dist = new_pos[order_array_plaquette[ix]]
            if dist[1] >= 2.5:
                dist[1] = -(size-dist[1])
    
            distance_plaquette_boundary[ix] = dist[1]
    

        boundary_vertex = np.around(distance_vertex_boundary+0.1)*0.5
        boundary_plaquette = np.around(distance_plaquette_boundary+0.1)*0.5

        boundary_v = np.zeros((len(boundary_vertex),2))
        boundary_v[:,0] = boundary_vertex

        boundary_p = np.ones((len(boundary_plaquette),2))
        boundary_p[:,0] = boundary_plaquette
    
        node_features = np.zeros((nbr_of_defects,2))
        node_features[order_array_vertex] = boundary_v
        node_features[order_array_plaquette] = boundary_p
    
    return node_features 


file_edge=open('defect_matrices/node_features.txt','a')

for ix in range(len(defects)):
    if (len(np.argwhere(defects[ix][0]))+len(np.argwhere(defects[ix][1]))) == 0:
        continue
    
    combined_matrix = combine_defect_matrices(defects[ix])
    node_features = distance_to_boundary(combined_matrix)

    np.savetxt(file_edge,node_features,fmt='%.1f')
    #file_edge.write("\n")

file_edge.close()

9
